In [16]:
%pip install svgpathtools

Note: you may need to restart the kernel to use updated packages.


In [17]:
from svgpathtools import svg2paths

paths, attributes = svg2paths('school-bus-2-scaled.svg')

In [ ]:
from svgpathtools import wsvg

# Assuming `puzzle_paths` is a list of path objects for each puzzle piece
wsvg(paths, attributes=attributes, filename='puzzle_output.svg')

In [54]:
import os

directory = 'puzzle_pieces'

# Get all the files in the directory
files = os.listdir(directory)

# Iterate over the files and delete them
for file in files:
    file_path = os.path.join(directory, file)
    os.remove(file_path)


In [52]:
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000
import numpy as np

def calculate_proximity(path1, path2):
    # This is a placeholder function; actual implementation depends on your specific needs
    # For example, you might calculate the minimum distance between the endpoints of the two paths
    return np.linalg.norm(np.array(path1.end) - np.array(path2.end))

def compare_colors(color1, color2):
    # Define how 'none' values are handled
    if color1 == 'none' and color2 == 'none':
        return True  # Consider 'none' to be equal to itself
    elif 'none' in [color1, color2]:
        return False  # If only one color is 'none', consider them different
    
    # Convert hex colors to LabColor for comparison (skip if either color is 'none')
    color1_lab = convert_color(sRGBColor.new_from_rgb_hex(color1), LabColor)
    color2_lab = convert_color(sRGBColor.new_from_rgb_hex(color2), LabColor)
    
    # Calculate the Delta E (CIE2000)
    delta_e = delta_e_cie2000(color1_lab, color2_lab)
    
    # Threshold for color similarity; adjust based on your requirements
    threshold = 5
    return delta_e < threshold

def should_combine(path1, path2, attr1, attr2):
    proximity_threshold = 10  # Adjust based on your SVG's scale
    length_threshold = 10  # Minimum path length to consider for combination
    
    if path1.length() < length_threshold or path2.length() < length_threshold:
        return False  # Don't combine very short paths
    
    if calculate_proximity(path1, path2) > proximity_threshold:
        return False  # Paths are too far apart
    
    # Adjust logic to handle 'none' values in color comparison
    if not compare_colors(attr1.get('stroke', 'none'), attr2.get('stroke', 'none')):
        return False  # Colors are too different
    
    return True
